In [1]:
######################################################
# Juypter
######################################################
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

from ioUtils import getFile, saveFile
import urllib
from webUtils import getHTML
from fsUtils import setFile, isFile
from fileUtils import getBaseFilename
from time import sleep
from searchUtils import findExt


######################################################
# Versions
######################################################
import sys
print("Python: {0}".format(sys.version))
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2021-02-13 20:38:38.494237


# Main

In [26]:
urls=["http://playback.fm/charts/top-100-songs", "http://playback.fm/charts/rock", "http://playback.fm/charts/country", 
      "http://playback.fm/charts/rnb", "https://playback.fm/charts/brasil"]
urlsX=["https://playback.fm/one-hit-wonders-1900s", "https://playback.fm/one-hit-wonders-1910s", "https://playback.fm/one-hit-wonders-1920s", 
       "https://playback.fm/one-hit-wonders-1930s", "https://playback.fm/one-hit-wonders-1940s", "https://playback.fm/one-hit-wonders-1950s", 
       "https://playback.fm/one-hit-wonders-1960s", "https://playback.fm/one-hit-wonders-1970s", "https://playback.fm/one-hit-wonders-1980s", 
       "https://playback.fm/one-hit-wonders-1990s", "https://playback.fm/one-hit-wonders-2000s", "https://playback.fm/one-hit-wonders-2010s"]
from fileUtils import getBaseFilename
from time import sleep
from fsUtils import setDir, setFile, isFile, mkDir
for url in urls:
    name = getBaseFilename(url)    
    savename="playback/{0}.p".format(name)
    if not isFile(savename):
        data, code = downloadURL(url)
        saveFile(idata=data, ifile=savename)
        sleep(3)
        continue

    saveDir = setDir("playback", name)
    #saveDir = mkDir("playback", name)
    print(saveDir,'\t',savename)
        
    bsdata = getHTML(savename)
    for ul in bsdata.findAll("ul", {"id": "menu-bar"}):
        print(len(ul.findAll("li")))
        for il,li in enumerate(ul.findAll("li")):
            ref = li.find('a')
            try:
                url = ref.attrs['href']
            except:
                continue
            isavename = setFile(saveDir, "{0}.p".format(ref.text))
            if isFile(isavename):
                continue

            fullURL = "https://playback.fm{0}".format(url)
            print(fullURL,'-->',isavename)
            data, code = downloadURL(fullURL)
            saveFile(idata=data, ifile=isavename)
            sleep(3)



playback/top-100-songs 	 playback/top-100-songs.p
129
playback/rock 	 playback/rock.p
68
playback/country 	 playback/country.p
79
playback/rnb 	 playback/rnb.p
81
playback/brasil 	 playback/brasil.p
126


In [30]:
for url in urls:
    name    = getBaseFilename(url)
    retval = {}
    saveDir = setDir("playback", name)
    files   = findExt(saveDir, ".p")
    for ifile in files:
        year   = getBaseFilename(ifile)
        retval[year] = []
        bsdata = getHTML(ifile)
        table  = bsdata.find("table", {"class": "chartTbl"}) # id="myTable">
        if table is not None:
            ths    = table.findAll("th")
            header = [th.text for th in ths]
            print(header)

        for itr,tr in enumerate(table.findAll("tr", {"itemprop": "track"})):
            print(itr,end='\t')
            tds = tr.findAll("td")
            artistData = tds[1]
            refArtist  = artistData.find('a', {"itemprop": "byArtist"})
            refName    = artistData.find('a', {"itemprop": "name"})

            artistName = None
            artistURL  = None
            if refArtist is not None:
                artistName = refArtist.text.replace("\n", "")
                artistURL  = refArtist.attrs['href']
            artistInfo = {"Name": artistName, "URL": artistURL}

            titleName = None
            titleURL  = None
            if refName is not None:
                titleName = refName.text.replace("\n", "")
                titleURL  = refName.attrs['href']
            titleInfo = {"Name": titleName, "URL": titleURL}

            songData = {"Artist": artistInfo, "Title": titleInfo}
            
            retval[year].append(songData)
            print(songData)
            
    savename = "/Volumes/Piggy/Charts/data/playback/results/{0}.".format(name)
    saveFile(idata=retval, ifile=savename)

['Rank', 'SongArtist', 'Title']
0	{'Artist': {'Name': 'Don McLean', 'URL': '/artist/don-mclean-top-songs'}, 'Title': {'Name': 'American Pie', 'URL': '/charts/top-100-songs/video/1972/Don-McLean-American-Pie'}}
1	{'Artist': {'Name': 'Hot Butter', 'URL': '/artist/hot-butter-top-songs'}, 'Title': {'Name': 'Popcorn', 'URL': '/charts/top-100-songs/video/1972/Hot-Butter-Popcorn'}}
2	{'Artist': {'Name': 'Harry Nilsson', 'URL': '/artist/harry-nilsson-top-songs'}, 'Title': {'Name': 'Without You', 'URL': '/charts/top-100-songs/video/1972/Harry-Nilsson-Without-You'}}
3	{'Artist': {'Name': 'The Moody Blues', 'URL': '/artist/the-moody-blues-top-songs'}, 'Title': {'Name': 'Nights in White Satin', 'URL': '/charts/top-100-songs/video/1972/The-Moody-Blues-Nights-in-White-Satin'}}
4	{'Artist': {'Name': 'Neil Young', 'URL': '/artist/neil-young-top-songs'}, 'Title': {'Name': 'Heart of Gold', 'URL': '/charts/top-100-songs/video/1972/Neil-Young-Heart-of-Gold'}}
5	{'Artist': {'Name': 'Roberta Flack', 'URL': 

['Rank', 'SongArtist', 'Title']
0	{'Artist': {'Name': 'Marion Harris', 'URL': '/artist/marion-harris-top-songs'}, 'Title': {'Name': "After You've Gone", 'URL': '/charts/top-100-songs/video/1919/Marion-Harris-After-Youve-Gone'}}
1	{'Artist': {'Name': 'Ben Selvin', 'URL': '/artist/ben-selvin-and-his-orchestra-top-songs'}, 'Title': {'Name': "I'm Forever Blowing Bubbles", 'URL': '/charts/top-100-songs/video/1919/Ben-Selvin-Im-Forever-Blowing-Bubbles'}}
2	{'Artist': {'Name': 'Henry Burr & Albert Campbell', 'URL': '/artist/henry-burr-top-songs'}, 'Title': {'Name': "I'm Forever Blowing Bubbles", 'URL': '/charts/top-100-songs/video/1919/Henry-Burr--Albert-Campbell-Im-Forever-Blowing-Bubbles'}}
3	{'Artist': {'Name': 'Al Jolson', 'URL': '/artist/al-jolson-top-songs'}, 'Title': {'Name': "I'll Say She Does", 'URL': '/charts/top-100-songs/video/1919/Al-Jolson-Ill-Say-She-Does'}}
4	{'Artist': {'Name': 'John Steel', 'URL': '/artist/john-steel-top-songs'}, 'Title': {'Name': 'A Pretty Girl is Like a Me

60	{'Artist': {'Name': 'Ludacris & Shaunna', 'URL': '/artist/ludacris-top-songs'}, 'Title': {'Name': 'Stand Up', 'URL': '/charts/top-100-songs/video/2003/Ludacris--Shaunna-Stand-Up'}}
61	{'Artist': {'Name': 'Sean Paul', 'URL': '/artist/sean-paul-top-songs'}, 'Title': {'Name': 'Like Glue', 'URL': '/charts/top-100-songs/video/2003/Sean-Paul-Like-Glue'}}
62	{'Artist': {'Name': 'John Mayer', 'URL': '/artist/john-mayer-top-songs'}, 'Title': {'Name': 'Your Body Is a Wonderland', 'URL': '/charts/top-100-songs/video/2003/John-Mayer-Your-Body-Is-a-Wonderland'}}
63	{'Artist': {'Name': 'Gareth Gates', 'URL': '/artist/gareth-gates-top-songs'}, 'Title': {'Name': 'Unchained Melody', 'URL': '/charts/top-100-songs/video/2003/Gareth-Gates-Unchained-Melody'}}
64	{'Artist': {'Name': 'Girls Aloud', 'URL': '/artist/girls-aloud-top-songs'}, 'Title': {'Name': 'Sound of the Underground', 'URL': '/charts/top-100-songs/video/2003/Girls-Aloud-Sound-of-the-Underground'}}
65	{'Artist': {'Name': 'Simply Red', 'URL'

['Rank', 'SongArtist', 'Title']
0	{'Artist': {'Name': 'Elvis Presley', 'URL': '/artist/elvis-presley-top-songs'}, 'Title': {'Name': 'Jailhouse Rock', 'URL': '/charts/top-100-songs/video/1957/Elvis-Presley-Jailhouse-Rock'}}
1	{'Artist': {'Name': 'Jerry Lee Lewis', 'URL': '/artist/jerry-lee-lewis-top-songs'}, 'Title': {'Name': 'Great Balls of Fire', 'URL': '/charts/top-100-songs/video/1957/Jerry-Lee-Lewis-Great-Balls-of-Fire'}}
2	{'Artist': {'Name': 'Buddy Holly', 'URL': '/artist/buddy-holly-top-songs'}, 'Title': {'Name': "That'll Be the Day", 'URL': '/charts/top-100-songs/video/1957/Buddy-Holly-Thatll-Be-the-Day'}}
3	{'Artist': {'Name': 'Paul Anka', 'URL': '/artist/paul-anka-top-songs'}, 'Title': {'Name': 'Diana', 'URL': '/charts/top-100-songs/video/1957/Paul-Anka-Diana'}}
4	{'Artist': {'Name': 'Elvis Presley', 'URL': '/artist/elvis-presley-top-songs'}, 'Title': {'Name': 'All Shook Up', 'URL': '/charts/top-100-songs/video/1957/Elvis-Presley-All-Shook-Up'}}
5	{'Artist': {'Name': 'Harry B

['Rank', 'SongArtist', 'Title']
0	{'Artist': {'Name': 'Fred Astaire & Leo Reisman', 'URL': '/artist/fred-astaire-top-songs'}, 'Title': {'Name': 'Night & Day', 'URL': '/charts/top-100-songs/video/1932/Fred-Astaire--Leo-Reisman-Night--Day'}}
1	{'Artist': {'Name': 'Bing Crosby', 'URL': '/artist/bing-crosby-top-songs'}, 'Title': {'Name': 'Brother, Can You Spare a Dime?', 'URL': '/charts/top-100-songs/video/1932/Bing-Crosby-Brother-Can-You-Spare-a-Dime'}}
2	{'Artist': {'Name': 'Rudy Vallee & his Connecticut Yankees', 'URL': '/artist/rudy-vallée-top-songs'}, 'Title': {'Name': 'Brother, Can You Spare a Dime?', 'URL': '/charts/top-100-songs/video/1932/Rudy-Vallee--his-Connecticut-Yankees-Brother-Can-You-Spare-a-Dime'}}
3	{'Artist': {'Name': 'Louis Armstrong', 'URL': '/artist/louis-armstrong-top-songs'}, 'Title': {'Name': 'All of Me', 'URL': '/charts/top-100-songs/video/1932/Louis-Armstrong-All-of-Me'}}
4	{'Artist': {'Name': 'Bing Crosby & The Mills Brothers', 'URL': '/artist/bing-crosby-top-so

...

['Rank', 'SongArtist', 'Title']
0	{'Artist': {'Name': 'Procol Harum', 'URL': '/artist/procol-harum-top-songs'}, 'Title': {'Name': 'A Whiter Shade of Pale', 'URL': '/charts/top-100-songs/video/1967/Procol-Harum-A-Whiter-Shade-of-Pale'}}
1	{'Artist': {'Name': 'The Beatles', 'URL': '/artist/the-beatles-top-songs'}, 'Title': {'Name': 'All You Need is Love', 'URL': '/charts/top-100-songs/video/1967/The-Beatles-All-You-Need-is-Love'}}
2	{'Artist': {'Name': 'The Monkees', 'URL': '/artist/the-monkees-top-songs'}, 'Title': {'Name': "I'm a Believer", 'URL': '/charts/top-100-songs/video/1967/The-Monkees-Im-a-Believer'}}
3	{'Artist': {'Name': 'The Beatles', 'URL': '/artist/the-beatles-top-songs'}, 'Title': {'Name': 'Hello, Goodbye', 'URL': '/charts/top-100-songs/video/1967/The-Beatles-Hello-Goodbye'}}
4	{'Artist': {'Name': 'Bee Gees', 'URL': '/artist/bee-gees-top-songs'}, 'Title': {'Name': '(The Lights Went Out In) Massachusetts', 'URL': '/charts/top-100-songs/video/1967/Bee-Gees-The-Lights-Went-O

['Rank', 'SongArtist', 'Title']
0	{'Artist': {'Name': 'The Beatles', 'URL': '/artist/the-beatles-top-songs'}, 'Title': {'Name': 'Get Back', 'URL': '/charts/top-100-songs/video/1969/The-Beatles-Get-Back'}}
1	{'Artist': {'Name': 'Archies', 'URL': '/artist/the-archies-top-songs'}, 'Title': {'Name': 'Sugar Sugar', 'URL': '/charts/top-100-songs/video/1969/Archies-Sugar-Sugar'}}
2	{'Artist': {'Name': 'Zager & Evans', 'URL': '/artist/zager-evans-top-songs'}, 'Title': {'Name': 'In the Year 2525 (Exordium & Terminus)', 'URL': '/charts/top-100-songs/video/1969/Zager--Evans-In-the-Year-2525-Exordium--Terminus'}}
3	{'Artist': {'Name': 'The Rolling Stones', 'URL': '/artist/the-rolling-stones-top-songs'}, 'Title': {'Name': 'Honky Tonk Woman', 'URL': '/charts/top-100-songs/video/1969/The-Rolling-Stones-Honky-Tonk-Woman'}}
4	{'Artist': {'Name': 'Elvis Presley', 'URL': '/artist/elvis-presley-top-songs'}, 'Title': {'Name': 'Suspicious Minds', 'URL': '/charts/top-100-songs/video/1969/Elvis-Presley-Suspic

['Rank', 'SongArtist', 'Title']
0	{'Artist': {'Name': 'George Michael', 'URL': '/artist/george-michael-top-songs'}, 'Title': {'Name': 'Careless Whisper', 'URL': '/charts/top-100-songs/video/1984/George-Michael-Careless-Whisper'}}
1	{'Artist': {'Name': 'Stevie Wonder', 'URL': '/artist/stevie-wonder-top-songs'}, 'Title': {'Name': 'I Just Called to Say I Love You', 'URL': '/charts/top-100-songs/video/1984/Stevie-Wonder-I-Just-Called-to-Say-I-Love-You'}}
2	{'Artist': {'Name': 'Prince', 'URL': '/artist/prince-top-songs'}, 'Title': {'Name': 'When Doves Cry', 'URL': '/charts/top-100-songs/video/1984/Prince-When-Doves-Cry'}}
3	{'Artist': {'Name': 'Tina Turner', 'URL': '/artist/tina-turner-top-songs'}, 'Title': {'Name': "What's Love Got to Do With It?", 'URL': '/charts/top-100-songs/video/1984/Tina-Turner-Whats-Love-Got-to-Do-With-It'}}
4	{'Artist': {'Name': 'Cyndi Lauper', 'URL': '/artist/cyndi-lauper-top-songs'}, 'Title': {'Name': 'Girls Just Wanna Have Fun ', 'URL': '/charts/top-100-songs/vi

['Rank', 'SongArtist', 'Title']
0	{'Artist': {'Name': 'Hoagy Carmichael', 'URL': '/artist/hoagy-carmichael-top-songs'}, 'Title': {'Name': 'Stardust', 'URL': '/charts/top-100-songs/video/1927/Hoagy-Carmichael-Stardust'}}
1	{'Artist': {'Name': 'Gene Austin', 'URL': '/artist/gene-austin-top-songs'}, 'Title': {'Name': 'My Blue Heaven', 'URL': '/charts/top-100-songs/video/1927/Gene-Austin-My-Blue-Heaven'}}
2	{'Artist': {'Name': 'Frankie Trumbauer', 'URL': '/artist/frankie-trumbauer-top-songs'}, 'Title': {'Name': "Singin' the Blues", 'URL': '/charts/top-100-songs/video/1927/Frankie-Trumbauer-Singin-the-Blues'}}
3	{'Artist': {'Name': 'Jack Smith', 'URL': '/artist/whispering-jack-smith-top-songs'}, 'Title': {'Name': 'Me & My Shadow', 'URL': '/charts/top-100-songs/video/1927/Jack-Smith-Me--My-Shadow'}}
4	{'Artist': {'Name': 'Bix Beiderbecke', 'URL': '/artist/bix-beiderbecke-top-songs'}, 'Title': {'Name': 'In a Mist', 'URL': '/charts/top-100-songs/video/1927/Bix-Beiderbecke-In-a-Mist'}}
5	{'Arti

ValueError: Did not recognize extension format .

['Rank', 'SongArtist', 'Title']


0	{'Artist': {'Name': 'Lady Gaga', 'URL': '/artist/lady-gaga-top-songs'}, 'Title': {'Name': 'Just Dance', 'URL': '/charts/rnb/video/2008/lady-gaga-just-dance'}}
1	{'Artist': {'Name': 'Beyoncé', 'URL': '/artist/beyoncé-top-songs'}, 'Title': {'Name': 'Single Ladies (Put a Ring on It)', 'URL': '/charts/rnb/video/2008/beyonce-single-ladies-put-a-ring-on-it'}}
2	{'Artist': {'Name': 'Alicia Keys', 'URL': '/artist/alicia-keys-top-songs'}, 'Title': {'Name': "Like You'll Never See Me Again", 'URL': '/charts/rnb/video/2008/alicia-keys-like-youll-never-see-me-again'}}
3	{'Artist': {'Name': 'Keyshia Cole', 'URL': '/artist/keyshia-cole-top-songs'}, 'Title': {'Name': 'I Remember', 'URL': '/charts/rnb/video/2008/keyshia-cole-i-remember'}}
4	{'Artist': {'Name': 'Lil Wayne Featuring Static Major', 'URL': '/artist/lil-wayne-top-songs'}, 'Title': {'Name': 'Lollipop', 'URL': '/charts/rnb/video/2008/lil-wayne-featuring-static-major-lollipop'}}
5	{'Artist': {'Name': 'Usher Featuring Young Jeezy', 'URL': '/a

In [19]:
table = bsdata.find("table", {"class": "chartTbl"}) # id="myTable">
